### 세마포어

In [84]:
import threading
import time

class ThreadPool(object):
    def __init__(self):
        self.active = []
        self.lock = threading.Lock()
        
    def acquire(self, name):
        with self.lock:
            self.active.append(name)
            print("획득: {0} | 스레드 풀: {1}".format(name, self.active))
            
    def release(self, name):
        with self.lock:
            self.active.remove(name)
            print("반환: {0} | 스레드 풀: {1}".format(name, self.active))
            

def worker( semaphore, pool ):
    with semaphore:
        name = threading.currentThread().getName()
        pool.acquire(name)
        time.sleep(1)
        pool.release(name)
            

if __name__ == '__main__':
    threads = []
    pool = ThreadPool()
    semaphore = threading.Semaphore(4)
    for i in range(10):
        t = threading.Thread( target=worker, name="스레드 "+str(i), args=(semaphore,pool))
        t.start()
        threads.append(t)
        
    for t in threads:
        t.join()


획득: 스레드 0 | 스레드 풀: ['스레드 0']
획득: 스레드 1 | 스레드 풀: ['스레드 0', '스레드 1']
획득: 스레드 2 | 스레드 풀: ['스레드 0', '스레드 1', '스레드 2']
획득: 스레드 3 | 스레드 풀: ['스레드 0', '스레드 1', '스레드 2', '스레드 3']
반환: 스레드 0 | 스레드 풀: ['스레드 1', '스레드 2', '스레드 3']
획득: 스레드 4 | 스레드 풀: ['스레드 1', '스레드 2', '스레드 3', '스레드 4']
반환: 스레드 1 | 스레드 풀: ['스레드 2', '스레드 3', '스레드 4']
획득: 스레드 5 | 스레드 풀: ['스레드 2', '스레드 3', '스레드 4', '스레드 5']
반환: 스레드 3 | 스레드 풀: ['스레드 2', '스레드 4', '스레드 5']
반환: 스레드 2 | 스레드 풀: ['스레드 4', '스레드 5']
획득: 스레드 6 | 스레드 풀: ['스레드 4', '스레드 5', '스레드 6']
획득: 스레드 7 | 스레드 풀: ['스레드 4', '스레드 5', '스레드 6', '스레드 7']
반환: 스레드 4 | 스레드 풀: ['스레드 5', '스레드 6', '스레드 7']
획득: 스레드 8 | 스레드 풀: ['스레드 5', '스레드 6', '스레드 7', '스레드 8']
반환: 스레드 5 | 스레드 풀: ['스레드 6', '스레드 7', '스레드 8']
획득: 스레드 9 | 스레드 풀: ['스레드 6', '스레드 7', '스레드 8', '스레드 9']
반환: 스레드 6 | 스레드 풀: ['스레드 7', '스레드 8', '스레드 9']
반환: 스레드 7 | 스레드 풀: ['스레드 8', '스레드 9']
반환: 스레드 8 | 스레드 풀: ['스레드 9']
반환: 스레드 9 | 스레드 풀: []


### 생산자 소비자의 문제

In [85]:
import threading
        
def consumer( cond ):
    name = threading.currentThread().getName()
    print("{0} 시작".format(name))
    with cond:    
        print("{0} 대기".format(name))
        cond.wait()
        print("{0} 자원 소비".format(name))

def producer( cond ):
    name = threading.currentThread().getName()
    print("{0} 시작".format(name))
    with cond:    
        print("{0} 자원 생산 후 모든 소비자에게 알림".format(name))
        cond.notifyAll()

if __name__ == '__main__':
    condition = threading.Condition()
    consumer1 = threading.Thread( target=consumer, name="소비자1", args=(condition,))
    consumer2 = threading.Thread( target=consumer, name="소비자2", args=(condition,))
    producer =  threading.Thread( target=producer, name="생산자", args=(condition,))    
    
    consumer1.start()
    consumer2.start()
    producer.start()

소비자1 시작
소비자2 시작
소비자2 대기
소비자1 대기
생산자 시작
생산자 자원 생산 후 모든 소비자에게 알림
소비자2 자원 소비
소비자1 자원 소비


### 프로파일러

In [89]:
import cProfile
import time

def sleep():
    time.sleep(5)

def hello_world():
    print("Hello World!")
    
def main():
    sleep()
    hello_world()
    hello_world()
    
cProfile.run('main()')

Hello World!
Hello World!
         70 function calls in 5.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    5.000    5.000 <ipython-input-89-05019ee8f61e>:10(main)
        1    0.000    0.000    5.000    5.000 <ipython-input-89-05019ee8f61e>:4(sleep)
        2    0.000    0.000    0.000    0.000 <ipython-input-89-05019ee8f61e>:7(hello_world)
        1    0.000    0.000    5.000    5.000 <string>:1(<module>)
        5    0.000    0.000    0.000    0.000 iostream.py:197(schedule)
        4    0.000    0.000    0.000    0.000 iostream.py:309(_is_master_process)
        4    0.000    0.000    0.000    0.000 iostream.py:322(_schedule_flush)
        4    0.000    0.000    0.000    0.000 iostream.py:384(write)
        5    0.000    0.000    0.000    0.000 iostream.py:93(_event_pipe)
        5    0.000    0.000    0.000    0.000 socket.py:342(send)
        5    0.000    0.000    0.000    0.000 threa

### 스택 만들기

In [94]:
class Stack(object):
    def __init__(self):
        self.items = []
        
    def isEmpty(self):
        return not bool(self.items)
    
    def push(self, value):
        self.items.append(value)
        
    def pop(self):
        value = self.items.pop()
        if value is not None:
            return value
        else:
            print("Stack is empty.")
    
    def size(self):
        return len(self.items)
    
    def peek(self):
        if self.items:
            return self.items[-1]
        else:
            print("Stack is empty.")
            
    def __repr__(self):
        return repr(self.items)
    
if __name__ == "__main__":
    stack = Stack()
    print(stack)
    for i in range(10):
        stack.push(i)
    print(stack)
    for i in range(10):
        print("pop : {0}".format(stack.pop()))
    print("pop : {0}".format(stack.pop()))

[]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
pop : 9
pop : 8
pop : 7
pop : 6
pop : 5
pop : 4
pop : 3
pop : 2
pop : 1
pop : 0


IndexError: pop from empty list

In [95]:
class Node(object):
    def __init__(self, value=None, pointer=None ):
        self.value = value
        self.pointer = pointer

class Stack(object):
    def __init__(self):
        self.head = None
        self.count = 0
        
    def isEmpty(self):
        return not bool(self.head)
    
    def push(self, item):
        self.head = Node(item, self.head)
        self.count += 1
        
    def pop(self):
        if( self.count>0 and self.head ):
            node = self.head
            self.head = node.pointer
            self.count -= 1
            return node.value
        else:
            print("Stack is empty.")
    
    def size(self):
        return self.count
    
    def peek(self):
        if( self.count>0 and self.head ):
            return self.head.value
        else:
            print("Stack is empty.")
            
    def _printList(self):
        node = self.head
        while node:
            print(node.value, end=" ")
            node = node.pointer
        print()
    
if __name__ == "__main__":
    stack = Stack()
    print(stack)
    for i in range(10):
        stack.push(i)
    print(stack)
    for i in range(10):
        print("pop : {0}".format(stack.pop()))
    print("pop : {0}".format(stack.pop()))

pop : 9
pop : 8
pop : 7
pop : 6
pop : 5
pop : 4
pop : 3
pop : 2
pop : 1
pop : 0
Stack is empty.
pop : None


### Queue 만들기

In [97]:
class Queue(object):
    def __init__(self):
        self.items = []
        
    def isEmpty(self):
        return not bool(self.items)
    
    def enqueue(self, value):
        self.items.insert(0, value)
        
    def dequeue(self):
        value = self.items.pop()
        if value is not None:
            return value
        else:
            print("Queue is empty.")
    
    def size(self):
        return len(self.items)
    
    def peek(self):
        if self.items:
            return self.items[-1]
        else:
            print("Queue is empty.")
            
    def __repr__(self):
        return repr(self.items)
    
if __name__ == "__main__":
    queue = Queue()
    print(queue)
    for i in range(10):
        queue.enqueue(i)
    print(queue)
    for i in range(10):
        print("dequeue : {0}".format(queue.dequeue()))

[]
[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
dequeue : 0
dequeue : 1
dequeue : 2
dequeue : 3
dequeue : 4
dequeue : 5
dequeue : 6
dequeue : 7
dequeue : 8
dequeue : 9


In [98]:
class Node(object):
    def __init__(self, value=None, pointer=None ):
        self.value = value
        self.pointer = pointer

class LinkedQueue(object):
    def __init__(self):
        self.head = None
        self.tail= None
        self.count = 0
        
    def isEmpty(self):
        return not bool(self.head)

    def dequeue(self):
        if( self.head ):
            value = self.head.value
            self.head = self.head.pointer
            self.count -= 1
            return value
        else:
            print("Queue is empty.")
    
    
    def enqueue(self, value):
        node = Node(value)
        if not self.head:
            self.head = node
            self.tail = node
        else:
            if self.tail:
                self.tail.pointer = node
            self.tail = node
        self.count += 1
        
    def size(self):
        return self.count
    
    def peek(self):
        if( self.count>0 ):
            return self.head.value
        else:
            print("Queue is empty.")
            
    def print(self):
        node = self.head
        while node:
            print(node.value, end=" ")
            node = node.pointer
        print()
    
if __name__ == "__main__":
    queue = Queue()
    print(queue)
    for i in range(10):
        queue.enqueue(i)
    print(queue)
    for i in range(10):
        print("pop : {0}".format(queue.dequeue()))
    print("pop : {0}".format(queue.dequeue()))

[]
[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
pop : 0
pop : 1
pop : 2
pop : 3
pop : 4
pop : 5
pop : 6
pop : 7
pop : 8
pop : 9


IndexError: pop from empty list

### heapq 모듈

In [99]:
import heapq

list1 = [4, 6, 8, 1]
heapq.heapify(list1)
list1

[1, 4, 8, 6]

In [107]:
h = [] 
heapq.heappush( h, (1, 'food'))
heapq.heappush( h, (2, 'have fun'))
heapq.heappush( h, (3, 'work'))
heapq.heappush( h, (4, 'study'))
type(h)
h


[(1, 'food'), (2, 'have fun'), (3, 'work'), (4, 'study')]

In [108]:
list1

[1, 4, 8, 6]

In [109]:
heapq.heappop(list1)

1

In [110]:
heapq.heappop(list1)

4

In [111]:
heapq.heappop(list1)

6

### heap 구현

In [129]:
left = 7
i = 3
largest = ( False ) and left or i
print(largest)

3


In [130]:
class Heapify(object):
    def __init__(self, data=None):
        self.data = data or []
        for i in range(len(data)//2, -1, -1):
#             print(i)
            self.__max_heapify__(i)
            
    def parent(self, i):
        if i & 1:
            return i >> 1
        else:
            return (i>>1) - 1
        
    def left_child(self, i):
        return (i<<1) + 1
    
    def right_child(self, i):
        return (i<<1) + 2
    
    def __max_heapify__(self, i):
        largest = i
        left = self.left_child(i)
        right = self.right_child(i)
        n = len(self.data)
        
        largest = (left<n and self.data[left] > self.data[i]) and left or i
        largest = (right<n and self.data[right] > self.data[largest]) and right or largest
        
        if i is not largest:
            self.data[i] , self.data[largest] = self.data[largest], self.data[i]
            print(self.data)
            self.__max_heapify__(largest)

    def extract_max(self):
        n = len(self.data)
        max_element = self.data[0]
        self.data[0] = self.data[n-1]
        self.data = self.data[:n-1]
        self.__max_heapify__(0)

        return max_element 
    
    def insert(self, item):
        i = len(self.data)
        self.data.append(item)
        while( i != 0 ) and item > self.data[self.parent(i)]:
            
            self.data[i] = self.data[self.parent(i)]
            i = self.parent(i)
        self.data[i] = item
        
def test_heapify():
    l1 = [1, 2, 3, 4, 5, 6, 7]
    h = Heapify(l1)
    print(h.extract_max())
#     print(h.extract_max())
#     print(h.extract_max())
#     h.insert(9)
#     print(h.data)

if __name__ == "__main__":
    test_heapify()
    

[1, 2, 7, 4, 5, 6, 3]
[1, 5, 7, 4, 2, 6, 3]
[7, 5, 1, 4, 2, 6, 3]
[7, 5, 6, 4, 2, 1, 3]
[6, 5, 3, 4, 2, 1]
7
